In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from yt_comment_scraper_util import YoutubeCommentScraper, CommentScraperUtil
from datetime import datetime
load_dotenv()
import csv

In [2]:
api_key = os.getenv('SRIKAR_API_1')  # make sure you have your API key in the .env file
scraper = YoutubeCommentScraper(api_key)

In [3]:
start_date = '2024-01-01'
channel = 'CNN'
scrape_iteration = 6

script_dir = os.getcwd()

path_to_link_csv = os.path.join(script_dir,f'../data/Raw Data/{channel}_links.csv')
comment_csv_load_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration)}.csv')
comment_csv_save_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration+1)}.csv')
path_to_error_logs = os.path.join(script_dir,f'../data/Error Logs/{channel}_{str(datetime.now())}.csv')


most_recent_video_id, previous_comments_df = CommentScraperUtil.get_earliest_video_id(comment_csv_load_path)
video_ids_to_scrape = CommentScraperUtil.load_video_ids(path_to_link_csv, start_date, most_recent_video_id)


In [4]:
all_comments = []
errors = []
for count, video_id in enumerate(video_ids_to_scrape):
    print(f'Scraping video #{count} | Video Id: {video_id}')
    test = scraper.get_video_comments(video_id)
    comments_data, error =test
    if comments_data:
        all_comments.extend(comments_data)
    if error:
        print(f'Error for {video_id}')
        errors.append({'video_id': video_id, "error": error})
        if error ==  "Quota Exceeded":
            break
new_comments = pd.DataFrame(all_comments)
errors_df = pd.DataFrame(errors)

Scraping video #0 | Video Id: _bqMQ-qb4Ls
Scraping video #1 | Video Id: AlHWfYsujjI
Scraping video #2 | Video Id: TzHEIk6EzYw
Scraping video #3 | Video Id: r8nM_2WUiZA
Scraping video #4 | Video Id: 8Z6sjXW-cBE
Scraping video #5 | Video Id: 9Lm7kr6R9_c
Scraping video #6 | Video Id: JaBmM8FU1Mk
Scraping video #7 | Video Id: kwPZJZ_cj7U
Scraping video #8 | Video Id: vIHQCrZ87I8
Scraping video #9 | Video Id: a2hVX3oExAY
Scraping video #10 | Video Id: yCwdF8stDyg
Scraping video #11 | Video Id: _l0AGY48LqM
Scraping video #12 | Video Id: BWO3wLULnoY
Scraping video #13 | Video Id: E-8_b9YAuDw
Scraping video #14 | Video Id: Bkz7rAnwzqQ
Scraping video #15 | Video Id: 24TVsK5rl9E
Scraping video #16 | Video Id: SKVNK9xxAI4
Scraping video #17 | Video Id: hXodqmZGmqE
Scraping video #18 | Video Id: LmkYLIqjna0
Scraping video #19 | Video Id: Vt9TKfA5Ssk
Scraping video #20 | Video Id: cnIebtqz4FY
Scraping video #21 | Video Id: NE97INTi9WE
Scraping video #22 | Video Id: z6aXBMGUUvc
Scraping video #23 | 

In [6]:

comments_df = new_comments
comments_df.to_csv(comment_csv_save_path, quoting=csv.QUOTE_NONNUMERIC, escapechar='\\', index=False, encoding='utf-8')
errors_df.to_csv(path_to_error_logs, index=False)
